# Create openalex_institutions

Creates `openalex.institutions.openalex_institutions` with:
- BIGINT primary key (matches `openalex_authors` and `openalex_works`)
- `ids` struct with URLs for API consumers
- Hash-based `updated_date` that only changes when content changes

This table replaces `institutions_api` (no separate formatting layer).

Uses:
- `openalex.institutions.institutions` - Institution master table
- `openalex.institutions.institution_ancestors` - Lineage IDs
- `openalex.works.openalex_works` - For works counts, topics, summary stats

Creates:
- `openalex.institutions.openalex_institutions` - Final enriched institutions table
- `openalex.institutions.openalex_institutions_hash` - Hash snapshot for change detection

In [ ]:
%run ../utils/variables

In [ ]:
%sql
-- Snapshot current hashes before rebuild
CREATE OR REPLACE TABLE openalex.institutions.openalex_institutions_hash AS
SELECT id, updated_date,
  xxhash64(CONCAT_WS('|',
    CAST(id AS STRING),
    COALESCE(display_name, ''),
    COALESCE(ror, ''),
    COALESCE(type, ''),
    COALESCE(country_code, ''),
    COALESCE(CAST(geo.latitude AS STRING), ''),
    COALESCE(CAST(geo.longitude AS STRING), ''),
    COALESCE(geo.city, ''),
    COALESCE(geo.region, ''),
    COALESCE(geo.country, ''),
    COALESCE(homepage_url, ''),
    COALESCE(ids.wikipedia, ''),
    COALESCE(ids.wikidata, ''),
    COALESCE(ids.grid, ''),
    COALESCE(TO_JSON(display_name_acronyms), '[]'),
    COALESCE(TO_JSON(display_name_alternatives), '[]'),
    COALESCE(CAST(works_count AS STRING), '0'),
    COALESCE(CAST(cited_by_count AS STRING), '0'),
    COALESCE(TO_JSON(summary_stats), '{}'),
    COALESCE(TO_JSON(lineage), '[]'),
    COALESCE(CAST(is_super_system AS STRING), 'false'),
    COALESCE(TO_JSON(roles), '[]'),
    COALESCE(TO_JSON(repositories), '[]'),
    COALESCE(TO_JSON(associated_institutions), '[]'),
    COALESCE(TO_JSON(topics), '[]'),
    COALESCE(TO_JSON(counts_by_year), '[]')
    -- topic_share excluded (non-deterministic: depends on global corpus)
  )) AS content_hash
FROM openalex.institutions.openalex_institutions;

In [ ]:
%sql
-- Create openalex_institutions table
CREATE OR REPLACE TABLE openalex.institutions.openalex_institutions AS
WITH institution_lineage AS (
  SELECT
    institution_id,
    FILTER(lineage_ids, x -> NOT ARRAY_CONTAINS(SUPER_SYSTEM_INSTITUTIONS, x)) AS lineage_ids
  FROM openalex.institutions.institution_ancestors
),

wa AS (
  SELECT
    w.id,
    w.publication_year,
    w.publication_date,
    w.cited_by_count,
    w.open_access,
    w.topics,
    w.updated_date,
    w.created_date,
    EXPLODE_OUTER(w.authorships) AS a
  FROM openalex.works.openalex_works w
),
wai AS (
  SELECT
    id,
    publication_year,
    publication_date,
    cited_by_count,
    open_access,
    topics,
    updated_date,
    created_date,
    EXPLODE_OUTER(a.institutions) AS inst
  FROM wa
),
-- per-work, per-institution base
works_base AS (
  SELECT
    CAST(replace(inst.id, 'https://openalex.org/I', '') AS BIGINT) as institution_id,
    id AS work_id,
    COALESCE(publication_year, YEAR(publication_date)) AS pub_year,
    CAST(cited_by_count AS INT) AS cited_by_count,
    CAST(open_access.is_oa AS BOOLEAN) AS is_oa,
    topics AS work_topics,
    updated_date,
    created_date
  FROM wai
  WHERE inst.id IS NOT NULL
),
-- dedupe to unique (work, institution) grain for counting works
unique_work_institution_pairs AS (
  SELECT DISTINCT
    work_id,
    institution_id,
    pub_year,
    cited_by_count,
    is_oa
  FROM works_base
),
-- compact per-institution rollup for totals and summary_stats helpers
main_agg_pre AS (
  SELECT
    institution_id,
    CAST(COUNT(DISTINCT work_id) AS INT) AS works_count,
    CAST(SUM(cited_by_count) AS INT) AS cited_by_count,
    CAST(SUM(CASE WHEN is_oa THEN 1 ELSE 0 END) AS INT) AS oa_works_count,
    CAST(AVG(CASE WHEN pub_year >= YEAR(current_date()) - 2 THEN cited_by_count END) AS DOUBLE) AS two_year_mean,
    CAST(COUNT_IF(cited_by_count >= 10) AS INT) AS i10_index,
    SORT_ARRAY(
      TRANSFORM(
        FILTER(COLLECT_LIST(cited_by_count), x -> x IS NOT NULL),
        x -> CAST(x AS INT)
      ),
      false
    ) AS sorted_citations,
    MAX(updated_date) AS updated_date,
    CAST(MIN(created_date) AS DATE) AS created_date
  FROM works_base
  GROUP BY institution_id
),
-- topics + topic_share (same template as authors/sources)
topics_and_share AS (
  WITH awt AS (
    SELECT DISTINCT
      institution_id,
      work_id,
      CAST(t.id AS STRING) AS topic_id,
      t.display_name, t.subfield, t.field, t.domain, t.score
    FROM (
      SELECT institution_id, work_id, EXPLODE_OUTER(work_topics) AS t
      FROM works_base
      WHERE work_id IS NOT NULL
    )
    WHERE t.id IS NOT NULL
  ),
  counts AS (
    SELECT
      institution_id,
      topic_id,
      MAX_BY(display_name, score) AS display_name,
      MAX_BY(subfield, score) AS subfield,
      MAX_BY(field, score) AS field,
      MAX_BY(domain, score) AS domain,
      MAX(score) AS score,
      COUNT(DISTINCT work_id) AS cnt
    FROM awt
    GROUP BY GROUPING SETS ((institution_id, topic_id), (topic_id))
  ),
  with_totals AS (
    SELECT
      institution_id,
      topic_id,
      display_name, subfield, field, domain, score,
      cnt AS topic_count,
      MAX(CASE WHEN institution_id IS NULL THEN cnt END) OVER (PARTITION BY topic_id) AS topic_total
    FROM counts
  )
  SELECT
    institution_id,
    SLICE(ARRAY_SORT(
      COLLECT_LIST(
        STRUCT(
          topic_id AS id,
          display_name,
          CAST(topic_count AS INT) AS count,
          score,
          subfield,
          field,
          domain
        )
      ),
      (l, r) -> CASE
        WHEN l.count > r.count THEN -1
        WHEN l.count < r.count THEN 1
        WHEN l.id < r.id THEN -1
        WHEN l.id > r.id THEN 1
        ELSE 0
      END
    ),1,25) AS topics,
    SLICE(ARRAY_SORT(
      COLLECT_LIST(
        STRUCT(
          topic_id AS id,
          display_name,
          ROUND(CAST(topic_count AS DOUBLE) / NULLIF(topic_total, 0), 7) AS value,
          subfield,
          field,
          domain
        )
      ),
      (l, r) -> CASE
        WHEN l.value > r.value THEN -1
        WHEN l.value < r.value THEN 1
        WHEN l.id < r.id THEN -1
        WHEN l.id > r.id THEN 1
        ELSE 0
      END
    ),1,25) AS topic_share
  FROM with_totals
  WHERE institution_id IS NOT NULL
  GROUP BY institution_id
),

-- finalize summary_stats struct from helpers
main_agg AS (
  SELECT
    institution_id AS id,
    NAMED_STRUCT(
      '2yr_mean_citedness', COALESCE(two_year_mean, 0.0),
      'h_index',
        CAST(
          ARRAY_MAX(
            ZIP_WITH(
              sorted_citations,
              SEQUENCE(1, SIZE(sorted_citations)),
              (citation, rank) -> IF(citation >= rank, rank, 0)
            )
          ) AS INT
        ),
      'i10_index', COALESCE(i10_index, 0)
    ) AS summary_stats,
    works_count,
    cited_by_count,
    oa_works_count,
    updated_date,
    created_date
  FROM main_agg_pre
),

-- Roles CTE
institution_roles AS (
  WITH entity_links_expanded AS (
    SELECT 
      i.id AS institution_id,
      el.id_1,
      el.id_2
    FROM openalex.institutions.institutions i
    INNER JOIN openalex.mid.entity_link el 
      ON (el.id_1 = CONCAT('I', i.id) OR el.id_2 = CONCAT('I', i.id))
  ),
  all_entity_ids AS (
    -- Institution's own role
    SELECT 
      i.id AS institution_id,
      CONCAT('I', i.id) AS entity_id,
      'institution' AS role
    FROM openalex.institutions.institutions i
    
    UNION ALL
    
    -- Linked publisher/funder roles
    SELECT 
      institution_id,
      CASE 
        WHEN id_1 LIKE 'I%' THEN id_2 
        ELSE id_1 
      END AS entity_id,
      CASE 
        WHEN id_1 LIKE 'P%' OR id_2 LIKE 'P%' THEN 'publisher'
        WHEN id_1 LIKE 'F%' OR id_2 LIKE 'F%' THEN 'funder'
      END AS role
    FROM entity_links_expanded
    WHERE (id_1 LIKE 'P%' OR id_1 LIKE 'F%' OR id_2 LIKE 'P%' OR id_2 LIKE 'F%')
  ),
  roles_with_counts AS (
    SELECT 
      ae.institution_id,
      ae.role,
      CONCAT('https://openalex.org/', ae.entity_id) AS id,
      CASE 
        -- For institution: use works_count from main_agg (same source as API)
        WHEN ae.role = 'institution' THEN ma.works_count
        -- For publisher: join to publishers_api table
        WHEN ae.role = 'publisher' THEN p_api.works_count
        -- For funder: join to funders_api table
        WHEN ae.role = 'funder' THEN f_api.works_count
      END AS works_count
    FROM all_entity_ids ae
    -- Join to main_agg for institution works_count (same source as API)
    LEFT JOIN main_agg ma 
      ON ae.role = 'institution' AND ae.institution_id = ma.id
    -- Join to publishers_api for publisher works_count
    LEFT JOIN openalex.publishers.publishers_api p_api
      ON ae.role = 'publisher' AND ae.entity_id = REPLACE(p_api.id, 'https://openalex.org/', '')
    -- Join to funders_api for funder works_count
    LEFT JOIN openalex.funders.funders_api f_api
      ON ae.role = 'funder' AND ae.entity_id = REPLACE(f_api.id, 'https://openalex.org/', '')
  ),
  -- Deduplicate funders (keep one with highest works_count)
  deduplicated_roles AS (
    SELECT 
      institution_id,
      role,
      id,
      CAST(COALESCE(works_count, 0) AS INT) AS works_count,
      ROW_NUMBER() OVER (
        PARTITION BY institution_id, role 
        ORDER BY works_count DESC, id
      ) AS rn
    FROM roles_with_counts
  )
  SELECT 
    institution_id,
    COLLECT_LIST(
      STRUCT(role, id, works_count)
    ) AS roles
  FROM deduplicated_roles
  WHERE rn = 1 OR role != 'funder'  -- Keep all non-funders, but only top funder
  GROUP BY institution_id
),

-- Repositories - matching API format
repositories AS (
  SELECT 
    s.institution_id,
    COLLECT_LIST(
      STRUCT(
        CONCAT('https://openalex.org/S', s.id) AS id,
        s.display_name,
        -- For repositories: host_organization is the institution
        CONCAT('https://openalex.org/I', s.institution_id) AS host_organization,
        i.display_name AS host_organization_name,
        -- For repositories: host_organization_lineage contains just the institution
        ARRAY(CONCAT('https://openalex.org/I', s.institution_id)) AS host_organization_lineage
      )
    ) AS repositories
  FROM openalex.sources.sources s
  INNER JOIN openalex.institutions.institutions i 
    ON s.institution_id = i.id
  WHERE s.institution_id IS NOT NULL
    AND s.type = 'repository'
    AND s.merge_into_id IS NULL
  GROUP BY s.institution_id
),

-- Associated Institutions - matching API format, type computed from ror_types
associated_institutions AS (
  WITH ror_type_lookup AS (
    -- Replicate PROD logic: select MAX(type) when multiple types exist, BUT prefer non-funder
    SELECT 
      ror_id,
      LOWER(COALESCE(
        MAX(CASE WHEN LOWER(type) != 'funder' THEN type END),  -- Prefer non-funder types
        MAX(type)                                              -- Fallback to any type (including funder)
      )) AS ror_type,
      collect_set(type) AS types
    FROM openalex.institutions.ror_types
    GROUP BY ror_id
  )
  SELECT 
    i.id AS institution_id,
    COLLECT_LIST(
      STRUCT(
        CONCAT('https://openalex.org/I', related_i.id) AS id,
        CONCAT('https://ror.org/', rr.related_ror_id) AS ror,
        related_i.display_name,
        related_i.iso3166_code AS country_code,
        LOWER(rt.ror_type) AS type,  -- Use MAX(type) from ror_types, lowercased
        LOWER(rr.relationship_type) AS relationship
      )
    ) AS associated_institutions
  FROM openalex.institutions.institutions i
  INNER JOIN openalex.institutions.ror_relationships rr 
    ON REPLACE(i.ror_id, 'https://ror.org/', '') = rr.ror_id
  INNER JOIN openalex.institutions.institutions related_i 
    ON rr.related_ror_id = REPLACE(related_i.ror_id, 'https://ror.org/', '')
  LEFT JOIN ror_type_lookup rt
    ON rr.related_ror_id = rt.ror_id
  WHERE related_i.merge_into_id IS NULL
  GROUP BY i.id
),

ror_alternative_names AS (
  WITH joined as (
    SELECT ror_id, label as alt_name FROM openalex.institutions.ror_labels
    UNION ALL
    SELECT ror_id, alias as alt_name FROM openalex.institutions.ror_aliases
  )
  SELECT ror_id, collect_set(alt_name) as alt_names
  FROM joined
  GROUP BY ror_id
),

-- counts_by_year aggregation
counts_by_year_agg AS (
  SELECT
    institution_id,
    FILTER(SORT_ARRAY(
      COLLECT_LIST(
        STRUCT(
          year,
          works_count,
          oa_works_count,
          cited_by_count
        )
      ),
      false
    ), x -> x.year >= 2010) AS counts_by_year
  FROM (
    SELECT
      institution_id,
      CAST(pub_year AS INT) AS year,
      CAST(COUNT(DISTINCT work_id) AS INT) AS works_count,
      CAST(SUM(CASE WHEN is_oa THEN 1 ELSE 0 END) AS INT) AS oa_works_count,
      CAST(SUM(cited_by_count) AS INT) AS cited_by_count
    FROM unique_work_institution_pairs
    WHERE pub_year IS NOT NULL
    GROUP BY institution_id, CAST(pub_year AS INT)
  )
  GROUP BY institution_id
)

SELECT
  i.id,
  i.ror_id AS ror,
  i.display_name,
  i.iso3166_code AS country_code,
  i.type,
  ARRAY_SORT(
    TRANSFORM(
      ARRAY_COMPACT(CONCAT(ARRAY(i.id), COALESCE(il.lineage_ids, ARRAY()))), id -> CONCAT('https://openalex.org/I', id)
    )
  ) AS lineage,
  ARRAY_CONTAINS(SUPER_SYSTEM_INSTITUTIONS, i.id) as is_super_system,
  CONCAT('https://openalex.org/institution-types/', i.type) AS type_id,
  i.official_page AS homepage_url,
  i.image_url,
  i.image_thumbnail_url,
  i.display_name_acronyms,
  -- Return empty array instead of NULL
  COALESCE(
    ARRAY_DISTINCT(ARRAY_COMPACT(CONCAT(COALESCE(i.display_name_alternatives, ARRAY()), COALESCE(ra.alt_names, ARRAY())))),
    ARRAY()
  ) AS display_name_alternatives,

  COALESCE(ma.works_count, 0) AS works_count,
  COALESCE(ma.cited_by_count, 0) AS cited_by_count,

  -- ids struct with full URLs for API consumers
  STRUCT(
    CONCAT('https://openalex.org/I', i.id) AS openalex,
    i.ror_id AS ror,
    i.grid_id AS grid,
    i.wiki_page AS wikipedia,
    i.wikidata_id AS wikidata,
  ) AS ids,

  -- Add roles
  COALESCE(ir.roles, ARRAY()) AS roles,

  -- Add repositories
  COALESCE(repo.repositories, ARRAY()) AS repositories,

  STRUCT(
    i.city,
    i.geonames_city_id,
    i.region,
    i.iso3166_code AS country_code,
    i.country,
    i.latitude,
    i.longitude
  ) AS geo,

  /* topics & concepts */
  COALESCE(ts.topics, ARRAY()) AS topics,
  COALESCE(ts.topic_share, ARRAY()) AS topic_share,

  -- Add associated_institutions
  COALESCE(ai.associated_institutions, ARRAY()) AS associated_institutions,

  /* counts_by_year */
  COALESCE(cby.counts_by_year, ARRAY()) AS counts_by_year,

  /* summary_stats */
  COALESCE(ma.summary_stats,
           NAMED_STRUCT('2yr_mean_citedness', CAST(0.0 AS DOUBLE),
                        'h_index', CAST(0 AS INT),
                        'i10_index', CAST(0 AS INT))) AS summary_stats,

  CONCAT('https://api.openalex.org/works?filter=institutions.id:I', CAST(i.id AS STRING)) AS works_api_url,

  CAST(NULL AS TIMESTAMP) AS updated_date,
  TO_TIMESTAMP(i.created_date) AS created_date

FROM openalex.institutions.institutions i
LEFT JOIN main_agg ma ON i.id = ma.id
LEFT JOIN topics_and_share ts ON i.id = ts.institution_id
LEFT JOIN institution_lineage il ON i.id = il.institution_id
LEFT JOIN ror_alternative_names ra ON REPLACE(i.ror_id, 'https://ror.org/', '') = ra.ror_id
LEFT JOIN institution_roles ir ON i.id = ir.institution_id
LEFT JOIN repositories repo ON i.id = repo.institution_id
LEFT JOIN associated_institutions ai ON i.id = ai.institution_id
LEFT JOIN counts_by_year_agg cby ON i.id = cby.institution_id
WHERE i.merge_into_id IS NULL;

In [ ]:
%sql
-- Compare hashes and set updated_date only when content changed
WITH new_hashes AS (
  SELECT id,
    xxhash64(CONCAT_WS('|',
      CAST(id AS STRING),
      COALESCE(display_name, ''),
      COALESCE(ror, ''),
      COALESCE(type, ''),
      COALESCE(country_code, ''),
      COALESCE(CAST(geo.latitude AS STRING), ''),
      COALESCE(CAST(geo.longitude AS STRING), ''),
      COALESCE(geo.city, ''),
      COALESCE(geo.region, ''),
      COALESCE(geo.country, ''),
      COALESCE(homepage_url, ''),
      COALESCE(ids.wikipedia, ''),
      COALESCE(ids.wikidata, ''),
      COALESCE(ids.grid, ''),
      COALESCE(TO_JSON(display_name_acronyms), '[]'),
      COALESCE(TO_JSON(display_name_alternatives), '[]'),
      COALESCE(CAST(works_count AS STRING), '0'),
      COALESCE(CAST(cited_by_count AS STRING), '0'),
      COALESCE(TO_JSON(summary_stats), '{}'),
      COALESCE(TO_JSON(lineage), '[]'),
      COALESCE(CAST(is_super_system AS STRING), 'false'),
      COALESCE(TO_JSON(roles), '[]'),
      COALESCE(TO_JSON(repositories), '[]'),
      COALESCE(TO_JSON(associated_institutions), '[]'),
      COALESCE(TO_JSON(topics), '[]'),
      COALESCE(TO_JSON(counts_by_year), '[]')
      -- topic_share excluded (non-deterministic: depends on global corpus)
    )) AS content_hash
  FROM openalex.institutions.openalex_institutions
)
MERGE INTO openalex.institutions.openalex_institutions AS target
USING (
  SELECT n.id,
    CASE
      WHEN p.id IS NULL THEN CURRENT_TIMESTAMP()
      WHEN n.content_hash <> p.content_hash THEN CURRENT_TIMESTAMP()
      ELSE p.updated_date
    END AS new_updated_date
  FROM new_hashes n
  LEFT JOIN openalex.institutions.openalex_institutions_hash p ON n.id = p.id
) AS source
ON target.id = source.id
WHEN MATCHED THEN UPDATE SET target.updated_date = source.new_updated_date;

In [ ]:
%sql
-- Verification: Row count comparison
SELECT 'openalex_institutions' as tbl, COUNT(*) as cnt FROM openalex.institutions.openalex_institutions
UNION ALL
SELECT 'institutions_api', COUNT(*) FROM openalex.institutions.institutions_api;

In [ ]:
%sql
-- Verification: Check Harvard (I18014758)
SELECT * FROM openalex.institutions.openalex_institutions WHERE id = 18014758;